<a href="https://colab.research.google.com/github/Ananya432000/sql/blob/main/sql_databank_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Bank Management and Analysis Using SQL**

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [ ]:
cursor.executescript("""
CREATE TABLE regions (
  region_id INTEGER PRIMARY KEY,
  region_name TEXT
);

CREATE TABLE customer_transactions (
  customer_id INTEGER,
  txn_date TEXT,
  txn_type TEXT,
  txn_amount INTEGER
);

CREATE TABLE customer_nodes (
  customer_id INTEGER,
  region_id INTEGER,
  node_id INTEGER,
  start_date TEXT,
  end_date TEXT
);
""")

conn.commit()
print("✅ Tables created successfully")

✅ Tables created successfully


In [ ]:
cursor.executescript("""
INSERT INTO regions VALUES (1, 'Asia'), (2, 'Europe'), (3, 'America');

INSERT INTO customer_nodes VALUES
(1,1,101,'2020-01-01','2020-06-01'),
(2,2,102,'2020-02-15',NULL),
(3,3,103,'2020-03-10','2020-09-10');

INSERT INTO customer_transactions VALUES
(1,'2020-01-02','deposit',100),
(1,'2020-01-03','withdrawal',50),
(2,'2020-02-12','purchase',200),
(3,'2020-03-15','deposit',300);
""")
conn.commit()

**1. How many different nodes make up the Data Bank network?**

In [ ]:
pd.read_sql_query("""
SELECT COUNT(DISTINCT node_id) AS total_nodes
FROM customer_nodes;
""", conn)

,total_nodes
0,3


**How many nodes are there in each region?**

In [ ]:
pd.read_sql_query("""
SELECT r.region_name, COUNT(DISTINCT c.node_id) AS nodes_per_region
FROM customer_nodes c
JOIN regions r ON c.region_id = r.region_id
GROUP BY r.region_name;
""", conn)

,region_name,nodes_per_region
0,America,1
1,Asia,1
2,Europe,1


**Q3. How many customers are divided among the regions?**

In [ ]:
pd.read_sql_query("""
SELECT r.region_name, COUNT(DISTINCT c.customer_id) AS customers_in_region
FROM customer_nodes c
JOIN regions r ON c.region_id = r.region_id
GROUP BY r.region_name;
""", conn)

,region_name,customers_in_region
0,America,1
1,Asia,1
2,Europe,1


**Q4. Total transaction amount per region**

In [ ]:
pd.read_sql_query("""
SELECT r.region_name, SUM(t.txn_amount) AS total_amount
FROM customer_transactions t
JOIN customer_nodes c ON t.customer_id = c.customer_id
JOIN regions r ON c.region_id = r.region_id
GROUP BY r.region_name;
""", conn)

,region_name,total_amount
0,America,300
1,Asia,150
2,Europe,200


**Q5. Average time (days) to move clients to a new node**

In [ ]:
pd.read_sql_query("""
SELECT ROUND(AVG(JULIANDAY(end_date) - JULIANDAY(start_date)),2) AS avg_days_to_move
FROM customer_nodes
WHERE end_date IS NOT NULL;
""", conn)

,avg_days_to_move
0,168.0


**Q6. Unique count & total amount for each transaction type**

In [ ]:
pd.read_sql_query("""
SELECT txn_type,
       COUNT(DISTINCT customer_id) AS unique_customers,
       SUM(txn_amount) AS total_amount
FROM customer_transactions
GROUP BY txn_type;
""", conn)

,txn_type,unique_customers,total_amount
0,deposit,2,400
1,purchase,1,200
2,withdrawal,1,50


**Average number and size of past deposits**

In [ ]:
pd.read_sql_query("""
WITH deposit_summary AS (
  SELECT customer_id,
         COUNT(*) AS deposit_count,
         SUM(txn_amount) AS total_amount
  FROM customer_transactions
  WHERE txn_type = 'deposit'
  GROUP BY customer_id
)
SELECT ROUND(AVG(deposit_count),2) AS avg_num_of_deposits,
       ROUND(AVG(total_amount/deposit_count),2) AS avg_deposit_size
FROM deposit_summary;
""", conn)

,avg_num_of_deposits,avg_deposit_size
0,1.0,200.0


**Q8. Monthly active customers with >1 deposit and ≥1 purchase or withdrawal**

In [ ]:
pd.read_sql_query("""
WITH monthly_activity AS (
  SELECT customer_id,
         SUBSTR(txn_date,1,7) AS month,
         SUM(CASE WHEN txn_type='deposit' THEN 1 ELSE 0 END) AS deposits,
         SUM(CASE WHEN txn_type='purchase' THEN 1 ELSE 0 END) AS purchases,
         SUM(CASE WHEN txn_type='withdrawal' THEN 1 ELSE 0 END) AS withdrawals
  FROM customer_transactions
  GROUP BY customer_id, SUBSTR(txn_date,1,7)
)
SELECT month,
       COUNT(DISTINCT customer_id) AS active_customers
FROM monthly_activity
WHERE deposits > 1 AND (purchases >= 1 OR withdrawals >= 1)
GROUP BY month
ORDER BY month;
""", conn)

,month,active_customers
